In [51]:
import pandas as pd, json
from config_path import *
projects = pd.read_pickle(f"{PATH_CLEAN}projects_current.pkl")
x=projects.loc[(projects.thema_code=='MSCA')&(projects.destination_code.isin(['DN', 'PF']))&(~projects.panel_code.isnull()), ['abstract', 'acronym', 'free_keywords', 'project_id', 'panel_code',  'title', 'topic_code', 'topic_name']].drop_duplicates()
panel=x.panel_code.unique()
msca_key = json.load(open("data_files/msca_keywords.json", encoding='utf-8'))
msca_key=pd.json_normalize(msca_key)

In [57]:
def clean_keyword(keyword):
    import re
    # Convertir en minuscules
    keyword = keyword.lower().replace("&", "and")
    # Supprimer les caractères spéciaux
    keyword = re.sub(r'[^a-zA-Z0-9\s]', ' ', keyword)
    return keyword


In [59]:
from functions_shared import prep_str_col, work_csv
df = x[['project_id', 'panel_code', 'free_keywords', 'abstract']]

df.loc[~df.abstract.isnull(), 'abstract'] = df.loc[~df.abstract.isnull(), 'abstract'].apply(lambda x: clean_keyword(x))
df['free_keyw'] = df['free_keywords'].apply(lambda x: [clean_keyword(kw) for kw in str(x).split('|')])
df = df.explode('free_keywords')

msca_key['panel_keywords'] = msca_key['panel_keywords'].apply(lambda x: clean_keyword(x))
msca_key.columns  
    

#     for i in msca_keywords:
#         if i.get('panel_regroupement_code') == p:

#             for c, row in df.iterrows():
#                 str1 = row..lower()
#                 str2 = "chemistry che"

#                 score = fuzz.ratio(str1, str2)
#                 print(f"Match score: {score}")

C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\905271631.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['free_keyw'] = df['free_keywords'].apply(lambda x: [clean_keyword(kw) for kw in str(x).split('|')])


Index(['panel_regroupement_code', 'panel_regroupement_name', 'panel_code_1',
       'panel_name_1', 'panel_keywords'],
      dtype='object')

In [60]:
from fuzzywuzzy import fuzz
def fuzzy_match(word, panel_keywords_series):
    best_match = None
    highest_score = 0
    for panel_keyword in panel_keywords_series:
        score = fuzz.token_set_ratio(word, panel_keyword)
        if score > highest_score:
            highest_score = score
            best_match = panel_keyword
    return best_match if highest_score > 80 else None  # Seuil de correspondance

# Appliquer le matching flou
# df['matched_panel_keyword'] = df['free_keywords'].apply(lambda x: fuzzy_match(x, msca_key['panel_keywords']))

In [61]:
for pan in df.panel_code.unique():
    print(pan)
    m=msca_key.loc[msca_key.panel_regroupement_code==pan]
    d=df.loc[df.panel_code==pan]
    d['matched_panel_keyword'] = d['free_keywords'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))
    d['matched_abstract'] = d['abstract'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))
    globals()[f'{pan}'] = d



CHE


C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_panel_keyword'] = d['free_keywords'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))
C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_abstract'] = d['abstract'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))


SOC


C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_panel_keyword'] = d['free_keywords'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))
C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_abstract'] = d['abstract'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))


ENG


...

C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_panel_keyword'] = d['free_keywords'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))
C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_abstract'] = d['abstract'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))


ECO


C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_panel_keyword'] = d['free_keywords'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))
C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_abstract'] = d['abstract'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))


MAT


C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_panel_keyword'] = d['free_keywords'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))
C:\Users\zfriant\AppData\Local\Temp\ipykernel_26000\511969227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['matched_abstract'] = d['abstract'].apply(lambda x: fuzzy_match(x, m['panel_keywords']))


In [62]:
CHE

,project_id,panel_code,free_keywords,abstract,free_keyw,matched_panel_keyword,matched_abstract
0,101068827,CHE,PROTAC|DNMT inhibitors|Epigenetics|Multiple My...,the dnmtac mm project aims at the development ...,"[protac, dnmt inhibitors, epigenetics, multipl...",None,organic chemistry
14,101109200,CHE,Cheese|matrix effect|milk fat globule membrane...,the current global cheese market is valued at ...,"[cheese, matrix effect, milk fat globule membr...",None,None
21,101059439,CHE,Reaction Prediction|Machine Learning|Chemoenzy...,enzymatic transformations are characterized by...,"[reaction prediction, machine learning, chemoe...",None,natural product synthesis
61,101061458,CHE,ATPase|Enzyme Design|Shortest Path Maps|Mutage...,project atpconf aim to search for the conforma...,"[atpase, enzyme design, shortest path maps, mu...",None,molecular dynamics
68,101106997,CHE,Lytic Polysaccharide Monooxygenases|QM|MM meth...,as the eu is committed to becoming the world s...,"[lytic polysaccharide monooxygenases, qm, mm m...",None,environment chemistry
...,...,...,...,...,...,...,...
134587,101199480,CHE,nanoplatelets|colloidal quantum dots|colloidal...,through the syroco project my objective is to...,"[nanoplatelets, colloidal quantum dots, colloi...",None,physical chemistry
134992,101227305,CHE,Multivalent interactions|Glycan|Optical micros...,virtually all mammalian cells are covered with...,"[multivalent interactions, glycan, optical mic...",microscopy,molecular biology
135110,101064371,CHE,flow chemistry|heterogeneous catalysis|single-...,the increase in global energy consumption has ...,"[flow chemistry, heterogeneous catalysis, sing...",catalysis,None
135155,101202441,CHE,Mechanophores|Activated carboxylic acids|Polym...,actimech aims to establish a groundbreaking cl...,"[mechanophores, activated carboxylic acids, po...",None,organic chemistry


In [55]:
msca_key[(msca_key.panel_regroupement_code=='CHE')&(msca_key.panel_keywords.str.contains('aptase'))]

,panel_regroupement_code,panel_regroupement_name,panel_code_1,panel_name_1,panel_keywords


In [4]:
from rapidfuzz import fuzz

str1 = "lgi consulting".lower()
str2 = "lgi sustainable innovation"

score = fuzz.ratio(str1, str2)
print(f"Match score: {score}")

Match score: 50.0


In [64]:
from fuzzywuzzy import fuzz

# Liste de chaînes de caractères dans laquelle chercher
list_of_strings = [
    "Pomme de terre",
    "Jus d'orange",
    "Salade de fruits",
    "Tarte aux pommes",
    "Glace à la vanille"
]

# Chaîne de recherche composée de plusieurs mots
search_string = ["pomme fruit", "salade fruits", "glace chocolat"]

# Fonction pour trouver la meilleure correspondance
def find_best_match(search_string, list_of_strings):
    best_match = None
    highest_score = 0
    for s in list_of_strings:
        score = fuzz.token_set_ratio(search_string, s)
        if score > highest_score:
            highest_score = score
            best_match = s
    return best_match, highest_score

# Trouver la meilleure correspondance
best_match, score = find_best_match(search_string, list_of_strings)

print(f"La meilleure correspondance pour '{search_string}' est '{best_match}' avec un score de {score}.")


La meilleure correspondance pour '['pomme fruit', 'salade fruits', 'glace chocolat']' est 'Salade de fruits' avec un score de 90.


In [67]:
from fuzzywuzzy import fuzz

# Liste de chaînes de caractères dans laquelle chercher
list_of_strings = [
    "Pomme de terre",
    "Jus d'orange",
    "Salade de fruits",
    "Tarte aux pommes",
    "Glace à la vanille"
]

# Liste de chaînes de recherche
search_strings = ["pomme & fruit", "salade de fruits", "glace chocolat"]

# Fonction pour trouver la meilleure correspondance pour un seul item
def find_best_match(search_string, list_of_strings):
    best_match = None
    highest_score = 0
    for s in list_of_strings:
        score = fuzz.token_set_ratio(search_string, s)
        if score > highest_score:
            highest_score = score
            best_match = s
    return best_match, highest_score

# Trouver la meilleure correspondance pour chaque item dans search_strings
best_matches = {}
for search_string in search_strings:
    best_match, score = find_best_match(search_string, list_of_strings)
    best_matches[search_string] = (best_match, score)

# Afficher les résultats
for search_string, (best_match, score) in best_matches.items():
    print(f"La meilleure correspondance pour '{search_string}' est '{best_match}' avec un score de {score}.")


La meilleure correspondance pour 'pomme & fruit' est 'Pomme de terre' avec un score de 62.
La meilleure correspondance pour 'salade de fruits' est 'Salade de fruits' avec un score de 100.
La meilleure correspondance pour 'glace chocolat' est 'Glace à la vanille' avec un score de 53.


In [69]:
from fuzzywuzzy import fuzz

# List of strings to search within
list_of_strings = [
    "Pomme de terre",
    "Jus d'orange",
    "Salade de fruits",
    "Tarte aux pommes",
    "Glace à la vanille"
]

# List of search strings
search_strings = ["pomme fruit", "salade fruits", "glace chocolat, pomme poire", "jus orange", "jus"]

# Function to tokenize a string
def tokenize(string):
    return string.lower().split()

# Function to find the best match using tokenization
def find_best_match_with_tokenization(search_string, list_of_strings):
    best_match = None
    highest_score = 0
    search_tokens = tokenize(search_string)

    for s in list_of_strings:
        s_tokens = tokenize(s)
        # Calculate the fuzzy match score using token set ratio
        score = fuzz.token_set_ratio(search_string, s)
        if score > highest_score:
            highest_score = score
            best_match = s
    return best_match, highest_score

# Find the best match for each item in search_strings
best_matches = {}
for search_string in search_strings:
    best_match, score = find_best_match_with_tokenization(search_string, list_of_strings)
    best_matches[search_string] = (best_match, score)

# Display the results
for search_string, (best_match, score) in best_matches.items():
    print(f"The best match for '{search_string}' is '{best_match}' with a score of {score}.")



The best match for 'pomme fruit' is 'Pomme de terre' with a score of 62.
The best match for 'salade fruits' is 'Salade de fruits' with a score of 100.
The best match for 'glace chocolat, pomme poire' is 'Pomme de terre' with a score of 53.
The best match for 'jus orange' is 'Jus d'orange' with a score of 100.
The best match for 'jus' is 'Jus d'orange' with a score of 100.
